# Nevronska Mreža

Spodaj je delna implementacija nevronske mreže, kot smo jo pričeli snovati na predavanjih. Manjka ključni del: implementacija algoritma za učenje. Za domačo nalogo bo tudi potreben popravek, ki mrežo prilagodi za večrazredne probleme (če se ne motim, bo ta potreben samo za eno vrstico v funkciji predict). Objekta za učenje in napovedovanje sta pisana skladno z zahtevami za tovrstne objekte v Orange-u, prilagoditve za sklearn vam ne bi smele vzeti dosti časa.

### Sigmoidna funkcija in dodajanje enic

In [1]:
import numpy as np
import Orange

def g(z):
    return 1/(1+np.exp(-z))

def add_ones(X):
    return np.column_stack((np.ones(len(X)), X))

### Learner

In [2]:
class NeuralNetLearner(Orange.classification.Learner):
    def __init__(self, arch, lambda_=1e-5):
        super().__init__()
        self.arch = arch
        self.lambda_ = lambda_
        self.name = "ann"

        self.theta_shape = np.array([(arch[i]+1, arch[i+1])
                                     for i in range(len(arch)-1)])
        ind = np.array([s1*s2 for s1, s2 in self.theta_shape])
        self.theta_ind = np.cumsum(ind[:-1])
        self.theta_len = sum(ind)

    def init_thetas(self, epsilon=1e-5):
        return np.random.rand(self.theta_len) * 2 * epsilon - epsilon

    def shape_thetas(self, thetas):
        t = np.split(thetas, self.theta_ind)
        return [t[i].reshape(shape) for i, shape in enumerate(self.theta_shape)]

    def h(self, a, thetas):
        """feed forward, prediction"""
        thetas = self.shape_thetas(thetas)
        for theta in thetas:
            a = g(add_ones(a).dot(theta))
        return a

    def J(self, thetas):
        # use matrix and vector operations. could be written in a single line
        # use self.m as stored by the fit function
        pass

    def grad_approx(self, thetas, e=1e-1):
        return np.array([(self.J(thetas+eps) - self.J(thetas-eps))/(2*e)
                         for eps in np.identity(len(thetas)) * e])

    def backprop(self, thetas):
        # cca 10 lines of code
        pass

    def fit(self, X, y, W=None):
        self.X, self.y = X, y
        self.m = self.X.shape[0]
        thetas = self.init_thetas()

        thetas, fmin, info = fmin_l_bfgs_b(self.J, thetas, self.backprop,
                                           callback=self.callback,
                                           factr=10)

        model = NeuralNetClassifier(self.domain, thetas)
        model.h = self.h
        return model

    def test(self, a):
        thetas = np.array([-30, 10, 20, -20, 20, -20, -10, 20, 20])
        print(self.h(a, thetas))

### Classifier

In [3]:
class NeuralNetClassifier(Orange.classification.Model):
    """Neural network classifier based on a set of binary classifiers."""
    def __init__(self, domain, thetas):
        super().__init__(domain)
        self.thetas = thetas  # model parameters

    def predict(self, X):
        y_hat = np.ravel(self.h(X, self.thetas))
        # following works only for binary classifiers, correct it for multiclass
        return np.vstack((1-y_hat, y_hat)).T

### Testiranje

Ne pozabite preveriti vsako vrstico, ki jo napišete. Prav tako ne priporočam, da vašo kodo za učenje pišete neposredno v zgornje ogrodje. Najprej skušajte kodo za učenje spisati vrstico za vrstico v Pythonovski lupini, preglejte, če so vaše matrike in vektorji pričakovanih dimenzij. Vse skupaj najprej implementirajte za mrežo z enim samim skritim nivojem in šele potem, ko stvar delaju, razvijte splošno različico za poljubno število nivojev.

Ne pozabite skalirati podatkov!

Enostavno testiranje z zelo majhnimi podatki bi bilo možno na primer z:

In [ ]:
data = Orange.data.Table("xor")
np.random.seed(42)
train, test = Orange.evaluation.sample(data, 5)

ann = NeuralNetLearner((data.X.shape[1], 5, 4), lambda_=0.1)

V končni različici bi lahko delovala tudi spodnja koda

In [ ]:
res = Orange.evaluation.CrossValidation(data, [ann], k=5)
print(Orange.evaluation.AUC(res))